<a href="https://colab.research.google.com/github/OliverRevilla/Spark_Pyspark/blob/main/Feature_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Spark Changes**

In [ ]:
# return spark version
spark.version

# return python version
import sys
sys.version_info

## **Verifying DataTypes**

In [ ]:
# Verifying DataTypes
"""
# create list of actual dtypes to check
actual_dtypes_list = df.dtypes
print(actual_dtypes_list)

# Iterate through the list of actual dtypes tuples
for attribute_tuple in actual_dtypes_list:
  
  # Check if column name is dictionary of expected dtypes
  col_name = attribute_tuple[0]
  if col_name in validation_dict:

    # Compare attribute types
    col_type = attribute_tuple[1]
    if col_type == validation_dict[col_name]:
      print(col_name + ' has expected dtype.')
"""

"\n# create list of actual dtypes to check\nactual_dtypes_list = df.dtypes\nprint(actual_dtypes_list)\n\n# Iterate through the list of actual dtypes tuples\nfor attribute_tuple in actual_dtypes_list:\n  \n  # Check if column name is dictionary of expected dtypes\n  col_name = attribute_tuple[0]\n  if col_name in validation_dict:\n\n    # Compare attribute types\n    col_type = attribute_tuple[1]\n    if col_type == validation_dict[col_name]:\n      print(col_name + ' has expected dtype.')\n"

## **Visually Inspecting Data**

In [ ]:
# df.describe(['Name_column','Name_column2',...]).show()
# pyspark.sql.functions.mean(col)
# pyspark.sql.functions.skewness(col)  #lack of symmetry
# pyspark.sql.functions.min(col)
# cov(col1,col2)
# corr(col1,col2)

# pyspark.sql.functions.mean(col)
df.agg({'SALESCLOSEPRICE':'mean'}).collect()

# cov(col1,col2)
df.cov('SALESCLOSEPRICE','YEARBUILT')

# Sample PySpark DataFrames before converting to Pandas
df.sample(False,0.5,42).count()
sns.distplot(a)

# Prepping for plotting a distribution
import seaborn as sns
sample_df = df.select(['SALESCLOSEPRICE']).sample(False,0.5,42) # 0.5 50% of the data, randomseed = 42
pandas_df = sample_df.toPandas()
sns.distplot(pandas_df)

# Checking the maximun correlation value of all columns with 'SALESCLOSEPRICE'
# Name and value of col with max corr
corr_max = 0
corr_max_col = columns[0]

# Loop to check all columns contained in list
for col in columns:
    # Check the correlation of a pair of columns
    corr_val = df.corr(col,'SALESCLOSEPRICE')
    # Logic to compare corr_max with current corr_val
    if corr_val > corr_max:
        # Update the column name and corr value
        corr_max = corr_val
        corr_max_col = col

print(corr_max_col)


# Select a single column and sample and convert to pandas
sample_df = df.select(['LISTPRICE']).sample(False, 0.5, 42)
pandas_df = sample_df.toPandas()
# Plot distribution of pandas_df and display plot
sns.distplot(pandas_df)
plt.show()
# Import skewness function
from pyspark.sql.functions import skewness
# Compute and print skewness of LISTPRICE
print(df.agg({'LISTPRICE': 'skewness'}).collect())

#Using Visualizations: Implot

# Select a the relevant columns and sample
sample_df = df.select(['SALESCLOSEPRICE', 'LIVINGAREA']).sample(False, 0.5, 42)

# Convert to pandas dataframe
pandas_df = sample_df.toPandas()

# Linear model plot of pandas_df
sns.lmplot(x= 'LIVINGAREA', y= 'SALESCLOSEPRICE', data= pandas_df)
plt.show()



# **Wrangling with Spark Functions**

**Dropping data**

In [ ]:
# List of columns to drop
cols_to_drop = ['NO','UNITNUMBER','CLASS']
# Drop columns
df = df.drop(*cols_to_drop)

# Show top 30 records
df.show(30)

# List of columns to remove from dataset
cols_to_drop = ['STREETNUMBERNUMERIC','LOTSIZEDIMENSIONS']

# Drop columns in list
df = df.drop(*cols_to_drop)

In [ ]:
# Text filtering
where(condicion)
like(expression)
isin(lista)
df = df.where(df['POTENCIALSHORTSALE'].like('Not Disclosed'))

# Inspect unique values in the column 'ASSUMABLEMORTGAGE'
df.select(['ASSUMABLEMORTGAGE']).distinct().show()

# List of possible values containing 'yes'
yes_values = ['Yes w/ Qualifying','Yes w/No Qualifying']

# Filter the text values out of df but keep null values
text_filter = ~df['ASSUMABLEMORTGAGE'].isin(yes_values) | df['ASSUMABLEMORTGAGE'].isNull()
df = df.where(text_filter)

# Print count of remaining records
print(df.count())

In [ ]:
# Value filtering

# Filtering outliers
std_val = df.agg({'SALESCLOSEPRICE':'stddev'}).collect()[0][0]
mean_val = df.agg({'SALESCLOSEPRICE':'mean'}).collect()[0][0]
hi_bound = mean_val + (3*std_val)
low_bound = mean_val - (3*std_val)
df = df.where((df['LISTPRICE'] < hi_bound) & (df['LISTPRICE'] > low_bound))

from pyspark.sql.functions import mean,stddev

# Calculate values used for outlier filtering
mean_val = df.agg({'log_SalesClosePrice': 'mean'}).collect()[0][0]
stddev_val = df.agg({'log_SalesClosePrice': 'stddev'}).collect()[0][0]

# Create three standard deviation (μ ± 3σ) lower and upper bounds for data
low_bound = mean_val - (3 * stddev_val)
hi_bound = mean_val + (3 * stddev_val)

# Filter the data to fit between the lower and upper bounds
df = df.where((df['log_SalesClosePrice'] < hi_bound) & (df['log_SalesClosePrice'] > low_bound))

In [ ]:
# Dropping NA's or Nulls
#.dropna(how = , thresh = ,subset =)
DataFrame.dropna()
# Drop any records with NULL Values
df = df.dropna()
# Drop records if both LISTPRICE and SALESPRICE are NULL
df = df.dropna(how = 'all', subset = ['LISTPRICE','SALESCLOSEPRICE'])
# Drop records where at least two columns have NULL values
df = df.dropna(thresh = 2)

In [ ]:
# Removing Duplicates
.dropDuplicates()
# Entire DataFrame
df.dropDuplicates()
# Check only a columns list
df.dropDuplicates(['streetaddress'])

**Adjusting Data**

In [ ]:
# Scaling data
# Minmax scaling
max_days = df.agg({'DAYSONMARKET':'max'}).collect()[0][0]
min_days = df.agg({'DAYSONMARKET':'min'}).collect()[0][0]
df = df.withColumn("scaled_days",
                   (df['DAYSONMARKET']- min_days)/(max_days - min_days))
df[['scales_days']].show(5)

In [ ]:
# Standarization: Transform the data to standard normal distribution
mean_days = df.agg({'DAYSONMARKET':'mean'}).collect()[0][0]
stddev_days = df.agg({'DAYSONMARKET':'stddev'}).collect()[0][0]
df = df.withColumn('ztrams_days',
                   (df['DAYSONMARKET']- mean_days)/stddev_days)
df.agg({'ztrans_days':'mean'}).collect()
df.agg({'ztrans_days':'stddev'}).collect()

# Define max and min values and collect them
max_days = df.agg({'DAYSONMARKET': 'max'}).collect()[0][0]
min_days = df.agg({'DAYSONMARKET': 'min'}).collect()[0][0]

# Create a new column based off the scaled data
df = df.withColumn('percentage_scaled_days', 
                  round((df['DAYSONMARKET'] - min_days) / (max_days - min_days)) * 100)

# Calc max and min for new column
print(df.agg({'percentage_scaled_days': 'max'}).collect())
print(df.agg({'percentage_scaled_days': 'min'}).collect())

## Function to scale
def min_max_scaler(df, cols_to_scale):
  # Takes a dataframe and list of columns to minmax scale. Returns a dataframe.
  for col in cols_to_scale:
    # Define min and max values and collect them
    max_days = df.agg({col: 'max'}).collect()[0][0]
    min_days = df.agg({col: 'min'}).collect()[0][0]
    new_column_name = 'scaled_' + col
    # Create a new column based off the scaled data
    df = df.withColumn(new_column_name, 
                      (df[col] - min_days) / (max_days - min_days))
  return df
  
df = min_max_scaler(df, cols_to_scale)
# Show that our data is now between 0 and 1
df[['DAYSONMARKET', 'scaled_DAYSONMARKET']].show()

In [ ]:
# Log scaling
from pyspark.sql.functions import log
df = df.withColumn('log_SalesClosePrice',log(df['SALESCLOSEPRICE']))

from pyspark.sql.functions import log

# Compute the skewness
print(df.agg({'YEARBUILT': 'skewness'}).collect())

# Calculate the max year
max_year = df.agg({'YEARBUILT': 'max'}).collect()[0][0]

# Create a new column of reflected data
df = df.withColumn('Reflect_YearBuilt', (max_year + 1) - df['YEARBUILT'])

# Create a new column based reflected data
df = df.withColumn('adj_yearbuilt', 1 / log(df['Reflect_YearBuilt']))

**Working with missing data**

In [ ]:
# Assessing Missing Values
# isNull()
df.where(df['ROOF'].isNull()).count()

# Function to automate dropping
def column_dropper(df, threshold):
  # Takes a dataframe and threshold for missing values. Returns a dataframe.
  total_records = df.count()
  for col in df.columns:
    # Calculate the percentage of missing values
    missing = df.where(df[col].isNull()).count()
    missing_percent = missing / total_records
    # Drop column if percent of missing is more than threshold
    if missing_percent > threshold:
      df = df.drop(col)
  return df

# Drop columns that are more than 60% missing
df = column_dropper(df, 0.6)

In [ ]:
# Plotting Missing Values
import seaborn as sns
sub_df = df.select(['ROOMAREA1'])
sample_df = sub_df.sample(False,0.5,4)
pandas_df = sample_df.toPandas()
sns.heatmap(data = pandas.isnull())

# Sample the dataframe and convert to Pandas
sample_df = df.select(columns).sample(False, 0.1, 42)
pandas_df = sample_df.toPandas()

# Convert all values to T/F
tf_df = pandas_df.isnull()

# Plot it
sns.heatmap(data=tf_df)
plt.xticks(rotation=30, fontsize=10)
plt.yticks(rotation=0, fontsize=10)
plt.show()

# Set the answer to the column with the most missing data
answer = 'BACKONMARKETDATE'

In [ ]:
# Imputation of missing values
# fillna(value, subset = None): 
#value: the value to replace missing with
#subset: the list of column names to replace missing
# Replacing missing values with zero
df.fillna(0, subset = ['DAYSONMARKET'])

# Replacing with the mean value for that column
col_mean = df.agg({'DAYSONMARKET':'mean'}).collect()[0][0]
df.fillna(col_mean,subset = ['DAYSONMARKET'])

# Sample the dataframe and convert to Pandas
sample_df = df.select(columns).sample(False, 0.1, 42)
pandas_df = sample_df.toPandas()

# Convert all values to T/F
tf_df = pandas_df.isnull()

# Plot it
sns.heatmap(data=tf_df)
plt.xticks(rotation=30, fontsize=10)
plt.yticks(rotation=0, fontsize=10)
plt.show()

# Set the answer to the column with the most missing data
answer = 'BACKONMARKETDATE'

**Joins**

In [ ]:
# Pyspark DataFrame Joins
DataFrame.join(
    other, # Other DataFrame to merge
    on = None, # The keys to join on
    how = None # Type of join to perform (default is 'inner')
)

# Example:
hdf.show(2)
# Specify the condition
cond = [df['OFFMARKETDATE'] == hdf['dt']]
# Join two hdf onto df
df = df.join(hdf, on = cond, 'left')
# How many sales occurred on bank holidays
df.where(df['nm'].isNull()).count()

# Cast data types
walk_df = walk_df.withColumn('longitude', walk_df['longitude'].cast('double'))
walk_df = walk_df.withColumn('latitude', walk_df['latitude'].cast('double'))

# Round precision
df = df.withColumn('longitude', round('longitude', 5))
df = df.withColumn('latitude', round('latitude', 5))

# Create join condition
condition = [walk_df['latitude'] == df['latitude'], walk_df['longitude'] == df['longitude']]

# Join the dataframes together
join_df = df.join(walk_df, on=condition, how='left')
# Count non-null records from new field
print(join_df.where(~join_df['walkscore'].isNull()).count())

In [ ]:
# Spak SQL Join
# Register the dataframe as a temp table
df.createOrReplaceTempView("df")
hdf.createOrReplaceTempView('hdf')
# Write a SQL statement
sql_df = spark.sql("""
    SELECT * 
    FROM df
    LEFT JOIN hdf
    ON df.OFFMARKETDATE =  hdf.dt 
""")

# Register dataframes as tables
df.createOrReplaceTempView('df')
walk_df.createOrReplaceTempView('walk_df')

# SQL to join dataframes
join_sql = 	"""
			SELECT 
				*
			FROM df
			LEFT JOIN walk_df
			ON df.longitude = walk_df.longitude
			AND df.latitude = walk_df.latitude
			"""
# Perform sql join
joined_df = spark.sql(join_sql)

# Join on mismatched keys precision 
wrong_prec_cond = [walk_df['longitude'] == df_orig['longitude'], walk_df['latitude'] == df_orig['latitude']]
wrong_prec_df = df_orig.join(walk_df, on= wrong_prec_cond, how='left')

# Compare bad join to the correct one
print(wrong_prec_df.where(wrong_prec_df['walkscore'].isNull()).count())
print(correct_join_df.where(correct_join_df['walkscore'].isNull()).count())

# Create a join on too few keys
few_keys_cond = [df['longitude'] == walk_df['longitude']]
few_keys_df = df.join(walk_df, on=few_keys_cond, how='left')

# Compare bad join to the correct one
print("Record Count of the Too Few Keys Join Example: " + str(few_keys_df.count()))
print("Record Count of the Correct Join Example: " + str(correct_join_df.count()))

# **Feature generation**

In [ ]:
# Creating a new feature area, by multiplicating
df = df.column('TSQTF', (df['WIDTH']*df['LENGTH']))

# Diference two date columns
df = df.withColumn('DAYSONMARKET', datediff('OFFMARKETDATE','LISTDATE'))

# Check the python libraries FeatureTools and TSFresh

# ---- Exercises -----------------------------------------------------------------------------------------
# Lot size in square feet
acres_to_sqfeet = 43560
df = df.withColumn('LOT_SIZE_SQFT', df['ACRES']*acres_to_sqfeet)

# Create a new column YARD_SIZE
df = df.withColumn('YARD_SIZE',df['LOT_SIZE_SQFT'] - df['FOUNDATIONSIZE'])

# Corr of ACRES vs SALESCLOSEPRICE
print("Corr of ACRES vs SALESCLOSEPRICE: " + str(df.corr('ACRES','SALESCLOSEPRICE')))

# Corr of FOUNDATIONSIZE vs SALESCLOSEPRICE
print("Corr of FOUNDATIONSIZE vs SALESCLOSEPRICE: " + str(df.corr('FOUNDATIONSIZE', 'SALESCLOSEPRICE')))

# Corr of YARD_SIZE vs SALESCLOSEPRICE
print('Corr of YARD_SIZE vs SALESCLOSEPRICE: ' + str(df.corr('YARD_SIZE','SALESCLOSEPRICE')))

#-----------------------------------------------------------------------
# ASSESSED_TO_LIST
df = df.withColumn('ASSESSED_TO_LIST', (df['ASSESSEDVALUATION']/df['LISTPRICE']))
df[['ASSESSEDVALUATION', 'LISTPRICE', 'ASSESSED_TO_LIST']].show(5)
# TAX_TO_LIST
df = df.withColumn('TAX_TO_LIST',(df['TAXES']/df['LISTPRICE']))
df[['TAX_TO_LIST', 'TAXES', 'LISTPRICE']].show(5)
# BED_TO_BATHS
df = df.withColumn('BED_TO_BATHS',(df['BEDROOMS']/df['BATHSTOTAL']))
df[['BED_TO_BATHS', 'BEDROOMS', 'BATHSTOTAL']].show(5)

# --------------------------------------------------------------------------


Time features

In [ ]:
from pyspark.sql.functions import to_date

# Cast the data type to Date
# to_date()
df = df.withColumn('LISTDATE', to_date('LISTDATE'))

from pyspark.sql.functions import year_month
# Create a new column of year number
df = df.withColumn('LIST_YEAR',year('LISTDATE'))

# Create a new column of month number
df = df.withColumn('LIST_MONTH', month('LISTDATE'))

from pyspark.sql.functions import dayofmonth, weekofyear

# Create new columns of the day number within the month
df = df.withColumns('LIST_DAYOFMONTH', dayofmonth('LISTDATE'))

# Create new columns of the week number within the year
df = df.withColumn('LIST_WEEKOFYEAR', weekofyear('LISTDATE'))

from pyspark.sql.functions import datediff

# Calculate difference between two date fields
df.withColumn('DAYSONMARKET', datediff('OFFMARKETDATE','LISTDATE'))


from pyspark.sql.functions import lag
from pyspark.sql.window import Window

# Create Window
w = Window().orderBy(m_df['DATE'])
# Create lagged column
m_df = m_df.withColumn('MORTAGE-1wk', lag('MORTAGE', count = 1).over(w))

# ----- Exercises --------------------
# Import needed fucntions
from pyspark.sql.functions import to_date,dayofweek

# Convert to date type
df = df.withColumn('LISTDATE',to_date('LISTDATE'))

# Get the day of the week
df = df.withColumn('List_Day_of_Week',dayofweek('LISTDATE'))

# Sample and convert to pandas dataframe
sample_df = df.sample(False,0.5,42).toPandas()

# Plot caount plot of of day of week
sns.countplot(x='List_Day_of_Week', data = sample_df )
plt.show()
#--------------------------------------
from pyspark.sql.functions import year

# Initialize dataframes
df = real_estate_df
price_df = median_prices_df

# Create year column
df = df.withColumn('list_year', year('LISTDATE'))

# Adjust year to match
df = df.withColumn('report_year',(df['list_year'] - 1))

# Create join condition
condition = [df['CITY'] == price_df['City'], df['report_year'] == price_df['Year']]

# Join the dataframes together
df = df.join(price_df, on=condition, how= 'left')
# Inspect that new columns are available
df[['MedianHomeValue']].show()

# -----------------------------------------
from pyspark.sql.functions import lag, datediff, to_date
from pyspark.sql.window import Window

# Cast data type
mort_df = mort_df.withColumn('DATE', to_date('DATE'))

# Create window
w = Window().orderBy(mort_df['DATE'])
# Create lag column
mort_df = mort_df.withColumn('DATE-1', lag('DATE', count=1).over(w))

# Calculate difference between date columns
mort_df = mort_df.withColumn('Days_Between_Report', datediff('DATE', 'DATE-1'))
# Print results
mort_df.select('Days_Between_Report').distinct().show()

Extracting features

In [ ]:
# Extracting features to apply machine learning

# Creating a boolean column
from pyspark.sql.functions import when 
# boolean filters
find_under_8 = df['ROOF'].like('%Age 8 Years or Less%')
find_over_8 = df['ROOF'].like('%Age Over 8 Years%')

# boolean filter 2th method
df = df.withColumn('old',(when(find_over_8,1)
                          .when(find_under_8,0)
                          .otherwise(None)))
# ---------------------------------------------------------------------
# Import needed functions
from pyspark.sql.functions import when

# Create boolean conditions for string matches
has_attached_garage = df['GARAGEDESCRIPTION'].like('%Attached Garage%')
has_detached_garage = df['GARAGEDESCRIPTION'].like('%Detached Garage%')

# Conditional value assignment 
df = df.withColumn('has_attached_garage', (when(has_attached_garage, 1)
                                          .when(has_detached_garage, 0)
                                          .otherwise(None)))

# Inspect results
df[['GARAGEDESCRIPTION', 'has_attached_garage']].show(truncate=100)

In [ ]:
# Spliting Columns
from pyspark.sql.functions import split
split_col = split(df['ROOF'],',')
# Put the first value of the list into a new column
df = df.withColumn('Roof_Material',split_col.getItem(0))
# Inspect results
df[['ROOF','Roof_Material']].show(5, truncate = 100)

# ----------------------------------------------------------------
# Import needed functions
from pyspark.sql.functions import when

# Create boolean conditions for string matches
has_attached_garage = df['GARAGEDESCRIPTION'].like('%Attached Garage%')
has_detached_garage = df['GARAGEDESCRIPTION'].like('%Detached Garage%')

# Conditional value assignment 
df = df.withColumn('has_attached_garage', (when(has_attached_garage, 1)
                                          .when(has_detached_garage, 0)
                                          .otherwise(None)))

# Inspect results
df[['GARAGEDESCRIPTION', 'has_attached_garage']].show(truncate=100)

In [ ]:
# Explode & Pivot
from pyspark.sql.functions import split, explode,lit,coalesce,first
  # Process
    # Split the column on commas into a list
    df = df.withColumn('roof_list', split(df['ROOF'],','))

    # Explode list into new records for each value
    ex_df = df.withColumn('ex_roof_list', explode(df['roof_list']))

    # Creating a dummy column of constant value to pivot later
    ex_df = ex_df.withColumn('constant_val',lit(1))

    # Pivot the values into boolean columns
    piv_df = ex_df.groupBy('NO').pivot('ex_roof_list')\
              .agg(coalesce(first('constant_val')))

# -----------------------------------------------------------
from pyspark.sql.functions import coalesce, first

# Pivot 
piv_df = ex_df.groupBy('NO').pivot('ex_garage_list').agg(coalesce(first('constant_val')))

# Join the dataframes together and fill null
joined_df = df.join(piv_df, on='NO', how='left')

# Columns to zero fill
zfill_cols = piv_df.columns

# Zero fill the pivoted values
zfilled_df = joined_df.fillna(0, subset= zfill_cols)

Binarizing, Bucketing, Encoding

In [ ]:
from pyspark.ml.feature import Binarizer
# Cast the data type to double
df = df.withColumn('FIREPLACES',df['FIREPLACES'].cast('double'))
# Create binarizer transformer
bin = Binarizer(threshold = 0.0,inputCol = 'FIREPLACES', outputCol = 'FirePlaceT')
# Apply the transformer
df = bin.transform(df)
# Inspect the results
df[['FIREPLACES','FireplaceT']].show(3)
# --------------------------------------------------
from pyspark.ml.feature import Binarizer
# Create the transformer
binarizer = Binarizer(threshold = 5.0,inputCol= 'List_Day_of_Week', outputCol='Listed_On_Weekend')

# Apply the transformation to df
df = binarizer.transform(df)

# Verify transformation
df[['List_Day_of_Week', 'Listed_On_Weekend']].show()

In [ ]:
from pyspark.ml.feature import Bucketizer
# Define how to split data --- float('Inf'): infinitve value
splits = [0,1,2,3,4, float('Inf')]
# Create bucketing transformer
buck = Bucketizer(splits = splits, inputCol = 'BATHSTOTAL', outputCol = 'baths')
# Apply transformer
df = buck.transform(df)
# Inspect results
df[['BATHSTOTAL','baths']].show(4)
#---------------------------------------------------
from pyspark.ml.feature import Bucketizer

# Plot distribution of sample_df
sns.distplot(sample_df, axlabel='BEDROOMS')
plt.show()

# Create the bucket splits and bucketizer
splits = [0,1, 2, 3, 4, 5, float('Inf')]
buck = Bucketizer(splits=splits, inputCol= 'BEDROOOMS', outputCol= 'bedrooms')

# Apply the transformation to df: df_bucket
df_bucket = buck.transform(df)

# Display results
df_bucket[['BEDROOMS', 'bedrooms']].show()

In [ ]:
# One Hot Encoding
from pyspark.ml.feature import OneHotEncoder, StringIndexer
# Process
  # Create Indexer transformer
  stringIndexer = StringIndexer(inputCol = 'CITY', outputCol = 'City_Index')
  # Fit transformer
  model = stringIndexer.fit(df)
  # Apply transformer
  indexed = model.transform(df)

  # Create Encoder Transformer
  encoder = OneHotEncoder(inputCol = 'City_Index', outputCol = 'City_Vec')
  # Apply the encoder transformer
  encoded_df = encoder.transform(indexed)
  # Inspect results
  encoded_df[['City_Vec']].show(4)

# ------------------------------------------------------------------------------
from pyspark.ml.feature import OneHotEncoder, StringIndexer

# Map strings to numbers with string indexer
string_indexer = StringIndexer(inputCol='SCHOOLDISTRICTNUMBER', outputCol='School_Index')
indexed_df = string_indexer.fit(df).transform(df)

# Onehot encode indexed values
encoder = OneHotEncoder(inputCol='School_Index', outputCol='School_Vec')
encoded_df = encoder.transform(indexed_df)

# Inspect the transformation steps
encoded_df[['SCHOOLDISTRICTNUMBER', 'School_Index', 'School_Vec']].show(truncate=100)

# **Building a Model**

### **Regression**



1.   GeneralizedLinearRegression
2.   IsotonicRegression
3.   LinearRegression
4.   DecisionTreeRegression
5.   GBTRegression
6.   RandomForestRegression




**Train Test Splits for time Series**

In [ ]:
# Train and Test Splits for Time Series
# Create Variables for max and min dates in our dataset
max_date = df.agg({'OFFMKTDATE': 'max'}).collect()[0][0]
min_date = df.agg({'OFFMKTDATE': 'min'}).collect()[0][0]

# Find how many days our data spans
from pyspark.sql.functions import datediff
range_in_days = datediff(max_date, min_date)

# Find the date to split the dataset on
from pyspark.sql.functions import date_add
split_in_days = round(range_in_days * 0.8)
split_date = date_add(min_date, split_in_days)

# Split the data into 80% train, 20% test
train_df = df.where(df['OFFMKTDATE'] < split_date)
test_df = df.where(df['OFFMKTDATE'] >= split_date)\
            .where(df['LISTDATE'] >= split_date) # to make sure that dates belong to the selected range
# ------------------------------------------------
def train_test_split_date(df, split_col, test_days= 45):
  """Calculate the date to split test and training sets"""
  # Find how many days our data spans
  max_date = df.agg({split_col: 'max'}).collect()[0][0]
  min_date = df.agg({split_col: 'min'}).collect()[0][0]
  # Subtract an integer number of days from the last date in dataset
  split_date = max_date - timedelta(days= test_days)
  return split_date

# Find the date to use in spitting test and train
split_date = train_test_split_date(df,'OFFMKTDATE')
# Create Sequential Test and Training Sets
train_df = df.where(df['OFFMKTDATE'] < split_date) 
test_df = df.where(df['OFFMKTDATE'] >= split_date).where(df['LISTDATE'] <= split_date) 
#----------------------------------------------------------------------------------
from pyspark.sql.functions import datediff,to_date,lit

split_date = to_date(lit('2017-12-10'))
# Create Sequential Test set
test_df = df.where(df['OFFMKTDATE'] >= split_date).where(df['LISTDATE'] <= split_date)

# Create a copy of DAYSONMARKET to review later
test_df = test_df.withColumn('DAYSONMARKET_Original', test_df['DAYSONMARKET'])

# Recalculate DAYSONMARKET from what we know on our split date
test_df = test_df.withColumn('DAYSONMARKET', datediff(split_date,'LISTDATE'))

# Review the difference
test_df[['LISTDATE', 'OFFMKTDATE', 'DAYSONMARKET_Original', 'DAYSONMARKET']].show()


**Preparing for Random Forest Regression**

Pyspark Ml algorithms require that all the features are in a single column as a vector. We will need convert our columns for the Random Forest Regression do work. To continue it mention how does it do.



In [ ]:
# Import the Vector Assembler transformation
from pyspark.ml.feature import VectorAssembler

# Replace missing values
df = df.fillna(-1)

# Define the columns to be converted to vectors
features_cols = list(df.columns)

# Remove the dependent variable from the list
features_cols.remove('SALESCLOSEPRICE')

# Create the vector assembler transformer
vec = VectorAssembler(inputCols = features_cols, outputCol = 'features')

# Apply the vector transformer to data
df = vec.transform(df)

# Select only the feature vectors and the dependent variable
nl_ready_df = df.select(['SALESCLOSEPRICE','features'])

# Inspect Results
nl_ready_df.show(5)

# --------------------------------------------------------------------------------
obs_threshold = 30
cols_to_remove = list()
# Inspect first 10 binary columns in list
for col in binary_cols[0:10]:
  # Count the number of 1 values in the binary column
  obs_count = df.agg({col: 'sum'}).collect()[0][0]
  # If less than our observation threshold, remove
  if obs_count <= obs_threshold:
    cols_to_remove.append(col)
    
# Drop columns and print starting and ending dataframe shapes
new_df = df.drop(*cols_to_remove)

print('Rows: ' + str(df.count()) + ' Columns: ' + str(len(df.columns)))
print('Rows: ' + str(new_df.count()) + ' Columns: ' + str(len(new_df.columns)))
#----------------------------------------------------------------------------------
# Replace missing values
df = df.fillna(-1, subset=['WALKSCORE','BIKESCORE'])

# Create list of StringIndexers using list comprehension
indexers = [StringIndexer(inputCol=col, outputCol=col+"_IDX")\
            .setHandleInvalid("keep") for col in categorical_cols]
# Create pipeline of indexers
indexer_pipeline = Pipeline(stages=indexers)
# Fit and Transform the pipeline to the original data
df_indexed = indexer_pipeline.fit(df).transform(df)

# Clean up redundant columns
df_indexed = df_indexed.drop(*categorical_cols)
# Inspect data transformations
print(df_indexed.dtypes)
#--------------------------------------------------------------------------------

**Building a Model**

Basic Model Parameters



1.   featuresCol = "features" : Independent Variables
2.   labelCol = "label": Dependent Variable
1.   predictionCol = "prediction": Column where we will put our predictions
2.   seed = None : Seed to the random process

Process:


1.   Import RandomForestRegressor
2.   Set the RandomForestRegressor model
1.   Fit the model to the dataframe
2.   Make predictions with it.
1.   See the results.
2.   Evaluating the model.



In [ ]:
# Import RandomForestRegressor
from pyspark.ml.regression import RandomForestRegressor

# Initialize model with columns to utilize
rf = RandomForestRegressor(featuresCol = 'features',
                           labelCol ='SALESCLOSEPRICE',
                           predictionCol = "Prediction_Price",
                           seed = 42)

# Train model
model = rf.fit(train_df)

# Make predictions
predictions = model.transform(test_df)

# Inspect results
predictions.select("Prediction_Price","SALESCLOSEPRICE").show(5)

# Evaluating the model
from pyspark.ml.evaluation import RegressionEvaluator

# Select Columns to compute test error
evaluator = RegressionEvaluator(labelCol = 'SALESCLOSEPRICE',
                                predictionCol = 'Prediction_Price')

# Create evaluation metrics
rmse = evaluator.evaluate(prediction, {evaluator.metricName: "rmse"})
r2 = evaluator.evaluate(prediction, {evaluator.metricName: "r2"})

# Print model metrics
print('RMSE: ' + str(rmse))
print('R**2: '- str(r2))

#------------------------------------------------
from pyspark.ml.regression import GBTRegressor

# Train a Gradient Boosted Trees (GBT) model.
gbt = GBTRegressor(featuresCol= 'features',
                           labelCol='SALESCLOSEPRICE',
                           predictionCol="Prediction_Price",
                           seed= 42
                           )

# Train model.
model = gbt.fit(train_df)

# Evaluating the model
from pyspark.ml.evaluation import RegressionEvaluator

# Select columns to compute test error
evaluator = RegressionEvaluator(labelCol= 'SALESCLOSEPRICE', 
                                predictionCol = 'Prediction_Price')
# Dictionary of model predictions to loop over
models = {'Gradient Boosted Trees': gbt_predictions, 'Random Forest Regression': rfr_predictions}
for key, preds in models.items():
    
  # Create evaluation metrics
  rmse = evaluator.evaluate(preds,{evaluator.metricName:"rmse"})
  r2 = evaluator.evaluate(preds,{evaluator.metricName:"r2"})


  # Print Model Metrics
  print(key + ' RMSE: ' + str(rmse))
  print(key + ' R^2: ' + str(r2))

Importance of the variables

In [ ]:
import pandas as pd

# Convert feature importances to a pandas column
fi_df = pd.DataFrame(model.featureImportances.toArray(),
                     columns = ['importance'])

# Convert list of feature names to pandas column
fi_df['feature'] = pd.Series(feature_cols)

# Sort the data based on feature importance
fi_df.sort_values(by = ['importance'], ascending = False, inplace = True)

# Interpret results
fi_df.head(9)

#### Savings and loading the model

# Save the model
model.save('rfr_real_estate_model')

# Load the model
from pyspark.ml.regression import RandomForestRegressionModel
model = RandomForestRegressionModel.load('rfr_real_estate_model')

# -----------------------------------------------------------------
from pyspark.ml.evaluation import RegressionEvaluator

# Select columns to compute test error
evaluator = RegressionEvaluator(labelCol= 'SALESCLOSEPRICE', 
                                predictionCol = 'Prediction_Price')
# Dictionary of model predictions to loop over
models = {'Gradient Boosted Trees': gbt_predictions, 'Random Forest Regression': rfr_predictions}
for key, preds in models.items():
    
  # Create evaluation metrics
  rmse = evaluator.evaluate(preds,{evaluator.metricName:"rmse"})
  r2 = evaluator.evaluate(preds,{evaluator.metricName:"r2"})


  # Print Model Metrics
  print(key + ' RMSE: ' + str(rmse))
  print(key + ' R^2: ' + str(r2))
from pyspark.ml.regression import RandomForestRegressionModel


# Save model
model.save('rfr_no_listprice')

# Load model
loaded_model = RandomForestRegressionModel.load('rfr_no_listprice')

